In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, SimpleRNN, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 1st Dataset

In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import requests
import zipfile
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Bidirectional, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm

# Load dataset
file_path = r"C:\Users\User\Downloads\Dataset-1.xlsx"
df = pd.read_excel(file_path)

# Check available columns
if "ABSTRACT" not in df.columns:
    raise KeyError("Dataset must contain 'ABSTRACT' column for text.")

# Combine multiple categories into a single label
label_columns = ["Computer Science", "Physics", "Mathematics", "Statistics", "Quantitative Biology", "Quantitative Finance"]
df["label"] = df[label_columns].idxmax(axis=1)

# Drop missing values
df.dropna(inplace=True)

# Extract text and labels
texts = df["ABSTRACT"].astype(str).tolist()
labels = df["label"].tolist()

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

# Train-validation-test split (70:20:10)
X_train, X_temp, y_train, y_temp = train_test_split(texts, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=1/3, random_state=42)

# Tokenization & Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)
max_length = 50
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length)
X_val_pad = pad_sequences(X_val_seq, maxlen=max_length)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length)

# Download and Load GloVe Embeddings
glove_path = "glove.6B.100d.txt"
glove_url = "https://nlp.stanford.edu/data/glove.6B.zip"
glove_zip = "glove.6B.zip"



def download_glove(url, dest):
    if os.path.exists(dest):
        os.remove(dest)
        
    print("Downloading GloVe embeddings...")
    retries = 3
    for attempt in range(retries):
        try:
            response = requests.get(url, stream=True, timeout=10)
            response.raise_for_status()
            total_size = int(response.headers.get("content-length", 0))
            with open(dest, "wb") as f, tqdm(
                desc="Downloading", total=total_size, unit="B", unit_scale=True
            ) as bar:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
                    bar.update(len(chunk))
            print("Download Complete!")
            return
        except Exception as e:
            print(f"Attempt {attempt+1}/{retries} failed: {e}")
            if attempt == retries - 1:
                raise

download_glove(glove_url, glove_zip)

print("Extracting GloVe embeddings...")
with zipfile.ZipFile(glove_zip, "r") as zip_ref:
    zip_ref.extract("glove.6B.100d.txt")
os.remove(glove_zip)

# Load GloVe embeddings into dictionary
embedding_index = {}
with open(glove_path, "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coef = np.asarray(values[1:], dtype="float32")
        embedding_index[word] = coef

# Create Embedding Matrix
embedding_dim = 100
num_words = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print(X_train_pad.shape, y_train.shape, X_val_pad.shape, y_val.shape, X_test_pad.shape, y_test.shape)

Downloading: 100%|██████████| 862M/862M [03:52<00:00, 3.71MB/s]   


Download Complete!
Extracting GloVe embeddings...
(14680, 50) (14680,) (4194, 50) (4194,) (2098, 50) (2098,)


In [11]:

# Function to build models
def build_model(model_type):
    model = Sequential([
        Embedding(input_dim=num_words, output_dim=embedding_dim, input_length=max_length,
                  weights=[embedding_matrix], trainable=False)
    ])

    if model_type == "RNN":
        model.add(SimpleRNN(64, return_sequences=True))
    elif model_type == "LSTM":
        model.add(LSTM(64, return_sequences=True))
    elif model_type == "GRU":
        model.add(GRU(64, return_sequences=True))
    elif model_type == "Bi-LSTM":
        model.add(Bidirectional(LSTM(64, return_sequences=True)))

    model.add(LSTM(32))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Train and evaluate each model
models = ["RNN", "LSTM", "GRU", "Bi-LSTM"]
results = {}
trained_models = {}

for model_type in models:
    print(f"\nTraining {model_type} Model...\n")
    model = build_model(model_type)
    history = model.fit(X_train_pad, y_train, epochs=10, batch_size=16, validation_data=(X_val_pad, y_val))

    # Get final training accuracy
    train_acc = history.history['accuracy'][-1]
    print(f"{model_type} Final Training Accuracy: {train_acc:.4f}")

    # Evaluate on test data
    loss, test_acc = model.evaluate(X_test_pad, y_test)
    results[model_type] = {"Accuracy": test_acc}
    trained_models[model_type] = model  # Store trained model
    print(f"{model_type} Test Accuracy: {test_acc:.4f}")
    report=classification_report(y_test, np.argmax(model.predict(X_test_pad), axis=1), target_names=label_encoder.classes_)
    results[model_type]["Report"] = report


# Print final results
print("\nFinal Model Results:")
for model, result in results.items():
    print(f"\n{model} Accuracy: {result['Accuracy']:.4f}")
    print(f"Classification Report of {model}:\n{result['Report']}")


# Predict on test data using the last trained model
print("\nPredictions on first 5 test samples:")
final_model = trained_models[models[-1]]  # Use last trained model

test_preds = final_model.predict(X_test_pad[:5])
predicted_labels = np.argmax(test_preds, axis=1)



Training RNN Model...



c:\Users\User\anaconda3\envs\spacy_env\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 14s 13ms/step - accuracy: 0.6460 - loss: 1.0271 - val_accuracy: 0.6755 - val_loss: 0.9334
Epoch 2/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 13s 14ms/step - accuracy: 0.7219 - loss: 0.8218 - val_accuracy: 0.7275 - val_loss: 0.8068
Epoch 3/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - accuracy: 0.7380 - loss: 0.7582 - val_accuracy: 0.7258 - val_loss: 0.7910
Epoch 4/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - accuracy: 0.7542 - loss: 0.7104 - val_accuracy: 0.7196 - val_loss: 0.7989
Epoch 5/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 13s 15ms/step - accuracy: 0.7666 - loss: 0.6634 - val_accuracy: 0.7270 - val_loss: 0.7871
Epoch 6/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - accuracy: 0.7792 - loss: 0.6254 - val_accuracy: 0.7363 - val_loss: 0.7685
Epoch 7/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - accuracy: 0.7952 - loss: 0.5821 - val_accuracy: 0.7356 - val_loss: 0.7811
Epoch 8/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 22s 24ms/step - accuracy: 0.8034 - loss: 0.5469 - 

c:\Users\User\anaconda3\envs\spacy_env\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


918/918 ━━━━━━━━━━━━━━━━━━━━ 35s 35ms/step - accuracy: 0.6608 - loss: 0.9804 - val_accuracy: 0.7098 - val_loss: 0.8685
Epoch 2/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 36s 40ms/step - accuracy: 0.7326 - loss: 0.7918 - val_accuracy: 0.7251 - val_loss: 0.8086
Epoch 3/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 38s 42ms/step - accuracy: 0.7477 - loss: 0.7280 - val_accuracy: 0.7320 - val_loss: 0.7655
Epoch 4/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 43s 47ms/step - accuracy: 0.7635 - loss: 0.6807 - val_accuracy: 0.7358 - val_loss: 0.7695
Epoch 5/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 42s 45ms/step - accuracy: 0.7753 - loss: 0.6364 - val_accuracy: 0.7279 - val_loss: 0.7944
Epoch 6/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 37s 40ms/step - accuracy: 0.7901 - loss: 0.5970 - val_accuracy: 0.7477 - val_loss: 0.7322
Epoch 7/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 36s 39ms/step - accuracy: 0.8040 - loss: 0.5595 - val_accuracy: 0.7451 - val_loss: 0.7417
Epoch 8/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 29s 31ms/step - accuracy: 0.8178 - loss: 0.5202 - val_accurac

c:\Users\User\anaconda3\envs\spacy_env\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


918/918 ━━━━━━━━━━━━━━━━━━━━ 35s 34ms/step - accuracy: 0.6809 - loss: 0.9355 - val_accuracy: 0.7213 - val_loss: 0.8164
Epoch 2/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 30s 33ms/step - accuracy: 0.7406 - loss: 0.7567 - val_accuracy: 0.7368 - val_loss: 0.7544
Epoch 3/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 30s 32ms/step - accuracy: 0.7594 - loss: 0.6943 - val_accuracy: 0.7401 - val_loss: 0.7279
Epoch 4/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 29s 32ms/step - accuracy: 0.7750 - loss: 0.6464 - val_accuracy: 0.7444 - val_loss: 0.7302
Epoch 5/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 31s 34ms/step - accuracy: 0.7890 - loss: 0.6018 - val_accuracy: 0.7446 - val_loss: 0.7324
Epoch 6/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 29s 31ms/step - accuracy: 0.8037 - loss: 0.5623 - val_accuracy: 0.7520 - val_loss: 0.7167
Epoch 7/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 28s 30ms/step - accuracy: 0.8176 - loss: 0.5158 - val_accuracy: 0.7492 - val_loss: 0.7517
Epoch 8/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 27s 29ms/step - accuracy: 0.8341 - loss: 0.4699 - val_accurac

c:\Users\User\anaconda3\envs\spacy_env\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


918/918 ━━━━━━━━━━━━━━━━━━━━ 47s 45ms/step - accuracy: 0.6697 - loss: 0.9470 - val_accuracy: 0.7132 - val_loss: 0.8336
Epoch 2/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 40s 43ms/step - accuracy: 0.7317 - loss: 0.7734 - val_accuracy: 0.7148 - val_loss: 0.8282
Epoch 3/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 38s 42ms/step - accuracy: 0.7506 - loss: 0.7118 - val_accuracy: 0.7339 - val_loss: 0.7571
Epoch 4/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 42s 45ms/step - accuracy: 0.7681 - loss: 0.6646 - val_accuracy: 0.7465 - val_loss: 0.7460
Epoch 5/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 48s 52ms/step - accuracy: 0.7811 - loss: 0.6208 - val_accuracy: 0.7330 - val_loss: 0.7774
Epoch 6/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 41s 45ms/step - accuracy: 0.7975 - loss: 0.5725 - val_accuracy: 0.7496 - val_loss: 0.7468
Epoch 7/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 36s 40ms/step - accuracy: 0.8173 - loss: 0.5230 - val_accuracy: 0.7303 - val_loss: 0.7532
Epoch 8/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 42s 46ms/step - accuracy: 0.8361 - loss: 0.4667 - val_accurac

# 2nd Dataset

In [14]:
import pandas as pd
import numpy as np
import spacy
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load dataset
file_path = r"C:\Users\User\Downloads\A2D2.xlsx"
df = pd.read_excel(file_path)

df.columns = ['TITLE', 'CONTENT', 'DOMAIN']  # Ensure correct column names

# Ensure correct columns exist
assert set(['TITLE', 'CONTENT']).issubset(df.columns), df.columns

# Safely cast to string and handle NaNs
df['TITLE'] = df['TITLE'].astype(str)
df['CONTENT'] = df['CONTENT'].astype(str)

# Or, if you want to preserve NaNs as blanks rather than "nan" strings:
# df['TITLE'] = df['TITLE'].fillna('').astype(str)
# df['CONTENT'] = df['CONTENT'].fillna('').astype(str)

df['TEXT'] = df['TITLE'] + ' ' + df['CONTENT']
 # Combine title & content

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    doc = nlp(text.lower())
    return " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])

df['TEXT'] = df['TEXT'].apply(preprocess_text)

# Encode labels
label_encoder = LabelEncoder()
df['LABEL'] = label_encoder.fit_transform(df['DOMAIN'])

# Tokenization
MAX_NUM_WORDS = 10000  # Max words in tokenizer
MAX_SEQUENCE_LENGTH = 200  # Max length of each sequence
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(df['TEXT'])
sequences = tokenizer.texts_to_sequences(df['TEXT'])
word_index = tokenizer.word_index
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['LABEL'], test_size=0.2, random_state=42)

In [15]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(3141, 200)
(3141,)
(786, 200)
(786,)


In [18]:
# --- Additional Imports for Modeling ---
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, SimpleRNN, Bidirectional, Dense, Dropout, LSTM, Embedding
from tensorflow.keras.optimizers import Adam

# --- Create a Validation Set from the Test Set (Best Practice) ---
# This splits your 20% test set into 10% validation and 10% test
X_val, X_test, y_val, y_test = train_test_split(
    X_test, y_test, test_size=0.5, random_state=42
)

print(f"Training samples: {X_train.shape[0]}")
print(f"Validation samples: {X_val.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")

# --- Define Hyperparameters for Training ---
EPOCHS = 5
BATCH_SIZE = 32

# --- Calculate Class Weights to handle imbalanced data ---
unique_classes, class_counts = np.unique(y_train, return_counts=True)
print(f"Training data distribution: {dict(zip(label_encoder.inverse_transform(unique_classes), class_counts))}")

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=unique_classes,
    y=y_train
)
class_weight_dict = {cls: weight for cls, weight in zip(unique_classes, class_weights)}
print(f"Calculated class weights: {class_weight_dict}")


# --- Reusable Function to Build, Train, and Evaluate Models ---
def build_and_train_model(model_type):
    """A function that handles the complete lifecycle for a given model type."""
    model = Sequential(name=model_type)
    model.add(Embedding(
        input_dim=MAX_NUM_WORDS,
        output_dim=EMBEDDING_DIM,
        input_length=MAX_SEQUENCE_LENGTH
    ))
    
    if model_type == "LSTM":
        model.add(LSTM(128, return_sequences=True))
        model.add(Dropout(0.3))
        model.add(LSTM(64))
    elif model_type == "GRU":
        model.add(GRU(128, return_sequences=True))
        model.add(Dropout(0.3))
        model.add(GRU(64))
    elif model_type == "RNN":
        model.add(SimpleRNN(128, return_sequences=True))
        model.add(Dropout(0.3))
        model.add(SimpleRNN(64))
    elif model_type == "BiLSTM":
        model.add(Bidirectional(LSTM(64, return_sequences=True)))
        model.add(Dropout(0.3))
        model.add(Bidirectional(LSTM(32)))
        
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(len(unique_classes), activation='softmax'))

    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    model.summary()

    print(f"\n--- Training {model_type} Model ---")
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),  # Use the validation set for tuning
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        class_weight=class_weight_dict, # Apply class weights
        verbose=1
    )

    print(f"\n--- Evaluating {model_type} on the Final Test Set ---")
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy of {model_type}: {accuracy * 100:.2f}%")

    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)
    
    print(f"\nClassification Report for {model_type}:")
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))
    
    return model, history

# --- Loop to Train and Evaluate All Specified Models ---
models_to_train = ["LSTM", "GRU", "RNN", "BiLSTM"]
for model_name in models_to_train:
    build_and_train_model(model_name)


Training samples: 3141
Validation samples: 393
Test samples: 393
Training data distribution: {'Entertainment': 490, 'Healthcare': 720, 'Sports': 269, 'Technology': 1214, 'Tourism': 448}
Calculated class weights: {0: 1.2820408163265307, 1: 0.8725, 2: 2.3353159851301117, 3: 0.5174629324546952, 4: 1.402232142857143}


c:\Users\User\anaconda3\envs\spacy_env\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


--- Training BiLSTM Model ---
Epoch 1/5
99/99 ━━━━━━━━━━━━━━━━━━━━ 21s 160ms/step - accuracy: 0.7692 - loss: 0.6658 - val_accuracy: 0.9084 - val_loss: 0.1746
Epoch 2/5
99/99 ━━━━━━━━━━━━━━━━━━━━ 15s 152ms/step - accuracy: 0.9863 - loss: 0.0941 - val_accuracy: 1.0000 - val_loss: 0.0073
Epoch 3/5
99/99 ━━━━━━━━━━━━━━━━━━━━ 15s 155ms/step - accuracy: 0.9940 - loss: 0.0436 - val_accuracy: 0.9949 - val_loss: 0.0177
Epoch 4/5
99/99 ━━━━━━━━━━━━━━━━━━━━ 22s 224ms/step - accuracy: 0.9984 - loss: 0.0186 - val_accuracy: 0.9949 - val_loss: 0.0140
Epoch 5/5
99/99 ━━━━━━━━━━━━━━━━━━━━ 22s 222ms/step - accuracy: 0.9987 - loss: 0.0120 - val_accuracy: 0.9975 - val_loss: 0.0094

--- Evaluating BiLSTM on the Final Test Set ---
Test Accuracy of BiLSTM: 99.24%
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 165ms/step

Classification Report for BiLSTM:
               precision    recall  f1-score   support

Entertainment       1.00      0.97      0.99        79
   Healthcare       0.98      1.00      0.99        97
      